#Agents and Agentic Patterns part. 1
###By: Wilfredo Aaron Sosa Ramos

Resource: https://docs.llamaindex.ai/en/stable/module_guides/deploying/agents/usage_pattern/

In [1]:
!pip install --upgrade llama-index-core llama-index-agent-openai llama-index-readers-file llama-index-postprocessor-cohere-rerank llama-index-llms-openai llama-index-embeddings-openai unstructured[html]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.4/149.4 kB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 249.9/249.9 kB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.3/454.3 kB 36.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 298.0/298.0 kB 25.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 59.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 37.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.9/274.9 kB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 71.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 60.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 5.0 MB/s eta 0:00:00
   

In [6]:
import os
from google.colab import userdata

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')

## ReAct Agent

In [25]:
from llama_index.core.tools import FunctionTool
from llama_index.llms.openai import OpenAI
from llama_index.core.agent import ReActAgent


# define sample Tool
def multiply(a: int, b: int) -> int:
    """Multiple two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

# initialize llm
llm = OpenAI(model="gpt-4o-mini")

# initialize ReAct agent
agent_init = ReActAgent.from_tools([multiply_tool], llm=llm, verbose=True)

In [26]:
agent_init.chat("What is 2123 * 215123")

> Running step 39fd5fe8-7bc8-4098-9dbd-fa92f027a21e. Step input: What is 2123 * 215123
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: multiply
Action Input: {'a': 2123, 'b': 215123}
Observation: 456706129
> Running step 69608cea-91e9-412f-bac7-fd741094c62e. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: 2123 * 215123 = 456706129


AgentChatResponse(response='2123 * 215123 = 456706129', sources=[ToolOutput(content='456706129', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2123, 'b': 215123}}, raw_output=456706129, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

In [5]:
from llama_index.core.agent import AgentRunner

best_agent = AgentRunner.from_llm([multiply_tool], llm=llm, verbose=True)

In [6]:
best_agent.chat("What is 2123 * 215123")

Added user message to memory: What is 2123 * 215123
=== Calling Function ===
Calling function: multiply with args: {"a":2123,"b":215123}
Got output: 456706129



AgentChatResponse(response='The result of \\( 2123 \\times 215123 \\) is \\( 456706129 \\).', sources=[ToolOutput(content='456706129', tool_name='multiply', raw_input={'args': (), 'kwargs': {'a': 2123, 'b': 215123}}, raw_output=456706129, is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

##Defining Tools
###Query Engine Tools

In [7]:
!mkdir -p 'data/10k/'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf' -O 'data/10k/uber_2021.pdf'
!wget 'https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf' -O 'data/10k/lyft_2021.pdf'

--2024-12-17 23:19:09--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/uber_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1880483 (1.8M) [application/octet-stream]
Saving to: ‘data/10k/uber_2021.pdf’

data/10k/uber_2021. 100%[===================>]   1.79M  --.-KB/s    in 0.07s   

2024-12-17 23:19:10 (24.3 MB/s) - ‘data/10k/uber_2021.pdf’ saved [1880483/1880483]

--2024-12-17 23:19:10--  https://raw.githubusercontent.com/run-llama/llama_index/main/docs/docs/examples/data/10k/lyft_2021.pdf
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.111.133, 185.199.108.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connec

In [28]:
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex

lyft_docs = SimpleDirectoryReader(
    input_files=["./data/10k/lyft_2021.pdf"]
).load_data()
uber_docs = SimpleDirectoryReader(
    input_files=["./data/10k/uber_2021.pdf"]
).load_data()

In [29]:
lyft_index = VectorStoreIndex.from_documents(lyft_docs)
uber_index = VectorStoreIndex.from_documents(uber_docs)

In [30]:
from llama_index.core.agent import ReActAgent
from llama_index.core.tools import QueryEngineTool, ToolMetadata

# NOTE: lyft_index and uber_index are both SimpleVectorIndex instances
lyft_engine = lyft_index.as_query_engine(similarity_top_k=3)
uber_engine = uber_index.as_query_engine(similarity_top_k=3)

query_engine_tools = [
    QueryEngineTool(
        query_engine=lyft_engine,
        metadata=ToolMetadata(
            name="lyft_10k",
            description="Provides information about Lyft financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        )
    ),
    QueryEngineTool(
        query_engine=uber_engine,
        metadata=ToolMetadata(
            name="uber_10k",
            description="Provides information about Uber financials for year 2021. "
            "Use a detailed plain text question as input to the tool.",
        )
    ),
]

# initialize ReAct agent
financial_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [14]:
financial_agent.chat("Give me a summary about Lyft financials for year 2021.")

> Running step 9991356b-5640-4578-ad38-950271714d0d. Step input: Give me a summary about Lyft financials for year 2021.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: lyft_10k
Action Input: {'input': 'Give me a summary about Lyft financials for year 2021.'}
Observation: Lyft's financial performance in 2021 showed positive trends compared to the previous year. The company experienced a 36% increase in revenue, with a notable 49.2% growth in the number of Active Riders in the fourth quarter of 2021 compared to the same period in 2020. The net loss decreased by 42% from $1.8 billion in 2020 to $1.0 billion in 2021, which included a benefit from a pre-tax gain related to a transaction with Woven Planet. Additionally, Lyft achieved its first annual Adjusted EBITDA profitability in 2021, amounting to $92.9 million. The company ended the year with $2.3 billion in unrestricted cash and cash equivalents, ensuring sufficient li

AgentChatResponse(response="Lyft's financial performance in 2021 showed positive trends compared to the previous year. The company experienced a 36% increase in revenue, with a notable 49.2% growth in the number of Active Riders in the fourth quarter of 2021 compared to the same period in 2020. The net loss decreased by 42% from $1.8 billion in 2020 to $1.0 billion in 2021, which included a benefit from a pre-tax gain related to a transaction with Woven Planet. Additionally, Lyft achieved its first annual Adjusted EBITDA profitability in 2021, amounting to $92.9 million. The company ended the year with $2.3 billion in unrestricted cash and cash equivalents, ensuring sufficient liquidity for ongoing operations and strategic initiatives.", sources=[ToolOutput(content="Lyft's financial performance in 2021 showed positive trends compared to the previous year. The company experienced a 36% increase in revenue, with a notable 49.2% growth in the number of Active Riders in the fourth quarter 

In [15]:
agent.chat("Give me a summary about Uber financials for year 2021.")

> Running step 9754af40-9f38-4150-b934-c07ba97392cf. Step input: Give me a summary about Uber financials for year 2021.
Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: uber_10k
Action Input: {'input': 'Give me a summary about Uber financials for year 2021.'}
Observation: In 2021, Uber experienced significant growth in its financial performance. The company reported a 57% increase in revenue compared to the previous year, reaching $17.5 billion. This growth was primarily driven by the expansion of its Delivery business and an increase in Freight revenue following the acquisition of Transplace. Despite a net loss attributable to Uber Technologies, Inc. of $496 million, there was a notable 93% improvement year-over-year. The company also reported an Adjusted EBITDA loss of $774 million, showing improvement from the previous year. Additionally, Uber ended the year with $4.3 billion in cash and cash equivalents.
> Running s

AgentChatResponse(response='In 2021, Uber experienced significant growth in its financial performance, reporting a 57% increase in revenue compared to the previous year, reaching $17.5 billion. This growth was primarily driven by the expansion of its Delivery business and an increase in Freight revenue following the acquisition of Transplace. Despite a net loss attributable to Uber Technologies, Inc. of $496 million, there was a notable 93% improvement year-over-year. The company also reported an Adjusted EBITDA loss of $774 million, showing improvement from the previous year. Additionally, Uber ended the year with $4.3 billion in cash and cash equivalents.', sources=[ToolOutput(content='In 2021, Uber experienced significant growth in its financial performance. The company reported a 57% increase in revenue compared to the previous year, reaching $17.5 billion. This growth was primarily driven by the expansion of its Delivery business and an increase in Freight revenue following the ac

###Use other agents as Tools

Resource: https://docs.llamaindex.ai/en/stable/examples/index_structs/struct_indices/SQLIndexDemo/

####Text-to-SQL Guide (Query Engine + Retriever)


In [16]:
from IPython.display import Markdown, display
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
)

In [17]:
engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()

In [18]:
# create city SQL table
table_name = "city_stats"
city_stats_table = Table(
    table_name,
    metadata_obj,
    Column("city_name", String(16), primary_key=True),
    Column("population", Integer),
    Column("country", String(16), nullable=False),
)
metadata_obj.create_all(engine)

#####Define SQL Database


In [19]:
from llama_index.core import SQLDatabase
from llama_index.llms.openai import OpenAI

In [20]:
llm_for_sql_agent = OpenAI(temperature=0.1, model="gpt-4o-mini")

In [21]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])

In [22]:
sql_database = SQLDatabase(engine, include_tables=["city_stats"])
from sqlalchemy import insert

rows = [
    {"city_name": "Toronto", "population": 2930000, "country": "Canada"},
    {"city_name": "Tokyo", "population": 13960000, "country": "Japan"},
    {
        "city_name": "Chicago",
        "population": 2679000,
        "country": "United States",
    },
    {"city_name": "Seoul", "population": 9776000, "country": "South Korea"},
]
for row in rows:
    stmt = insert(city_stats_table).values(**row)
    with engine.begin() as connection:
        cursor = connection.execute(stmt)

In [23]:
# view current table
stmt = select(
    city_stats_table.c.city_name,
    city_stats_table.c.population,
    city_stats_table.c.country,
).select_from(city_stats_table)

with engine.connect() as connection:
    results = connection.execute(stmt).fetchall()
    print(results)

[('Toronto', 2930000, 'Canada'), ('Tokyo', 13960000, 'Japan'), ('Chicago', 2679000, 'United States'), ('Seoul', 9776000, 'South Korea')]


#####Query Index

In [24]:
from sqlalchemy import text

with engine.connect() as con:
    rows = con.execute(text("SELECT city_name from city_stats"))
    for row in rows:
        print(row)

('Chicago',)
('Seoul',)
('Tokyo',)
('Toronto',)


#####Part 1: Text-to-SQL Query Engine

In [25]:
from llama_index.core.query_engine import NLSQLTableQueryEngine

query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database, tables=["city_stats"], llm=llm_for_sql_agent
)
query_str = "Which city has the highest population?"
response = query_engine.query(query_str)

In [26]:
display(Markdown(f"<b>{response}</b>"))

<b>The city with the highest population is Tokyo, with a population of 13,960,000.</b>

#####Part 2: Query-Time Retrieval of Tables for Text-to-SQL

In [27]:
from llama_index.core.indices.struct_store.sql_query import (
    SQLTableRetrieverQueryEngine,
)
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import VectorStoreIndex

# set Logging to DEBUG for more detailed outputs
table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats"))
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
query_engine = SQLTableRetrieverQueryEngine(
    sql_database, obj_index.as_retriever(similarity_top_k=1)
)

In [28]:
response = query_engine.query("Which city has the highest population?")
display(Markdown(f"<b>{response}</b>"))

<b>Tokyo has the highest population among all cities, with a population of 13,960,000.</b>

In [29]:
# you can also fetch the raw result from SQLAlchemy!
response.metadata["result"]

[('Tokyo', 13960000)]

In [30]:
# manually set context text
city_stats_text = (
    "This table gives information regarding the population and country of a"
    " given city.\nThe user will query with codewords, where 'foo' corresponds"
    " to population and 'bar'corresponds to city."
)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    (SQLTableSchema(table_name="city_stats", context_str=city_stats_text))
]

#####Part 3: Text-to-SQL Retriever

In [31]:
from llama_index.core.retrievers import NLSQLRetriever

# default retrieval (return_raw=True)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=True
)

In [32]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [33]:
from llama_index.core.response.notebook_utils import display_source_node

for n in results:
    display_source_node(n)

**Node ID:** be96083e-9f71-4a54-9948-28e79bde53da<br>**Similarity:** None<br>**Text:** [('Tokyo', 13960000), ('Seoul', 9776000), ('Toronto', 2930000), ('Chicago', 2679000)]<br>

In [34]:
# default retrieval (return_raw=False)
nl_sql_retriever = NLSQLRetriever(
    sql_database, tables=["city_stats"], return_raw=False
)

In [35]:
results = nl_sql_retriever.retrieve(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [36]:
# NOTE: all the content is in the metadata
for n in results:
    display_source_node(n, show_source_metadata=True)

**Node ID:** 8a78168e-96f3-407a-8309-f947030f75eb<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Tokyo', 'population': 13960000}<br>

**Node ID:** 552ae0c7-561b-4d6f-8f48-04e95976a3f0<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Seoul', 'population': 9776000}<br>

**Node ID:** b15f7c52-3de7-4437-aee5-15f4cf0c2975<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Toronto', 'population': 2930000}<br>

**Node ID:** 67a35657-cda6-4f14-91e4-24a34d80b350<br>**Similarity:** None<br>**Text:** <br>**Metadata:** {'city_name': 'Chicago', 'population': 2679000}<br>

#####Plug into our RetrieverQueryEngine

In [37]:
from llama_index.core.query_engine import RetrieverQueryEngine

query_engine = RetrieverQueryEngine.from_args(nl_sql_retriever)

In [38]:
response = query_engine.query(
    "Return the top 5 cities (along with their populations) with the highest population."
)

In [39]:
print(str(response))

Tokyo: 13,960,000  
Seoul: 9,776,000  
Toronto: 2,930,000  
Chicago: 2,679,000


####SQL Agent
Resource: https://docs.llamaindex.ai/en/stable/examples/agent/agent_runner/query_pipeline_agent/?h=sql+agent#building-an-agent-around-a-query-pipeline

In [2]:
from llama_index.core import SQLDatabase
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
    select,
    column,
)

engine = create_engine("sqlite:///chinook.db")
sql_database = SQLDatabase(engine)

In [3]:
from llama_index.core.query_pipeline import QueryPipeline

In [46]:
!curl "https://www.sqlitetutorial.net/wp-content/uploads/2018/03/chinook.zip" -O ./chinook.zip
!unzip ./chinook.zip

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  298k  100  298k    0     0  1835k      0 --:--:-- --:--:-- --:--:-- 1842k
curl: (6) Could not resolve host: .
Archive:  ./chinook.zip
replace chinook.db? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
  inflating: chinook.db              


#####Setup ReAct Agent Pipeline

In [4]:
from llama_index.core.query_pipeline import QueryPipeline as QP

qp = QP(verbose=True)

#####Setup Text-to-SQL Query Engine / Tool

In [7]:
from llama_index.core.query_engine import NLSQLTableQueryEngine
from llama_index.core.tools import QueryEngineTool

sql_query_engine = NLSQLTableQueryEngine(
    sql_database=sql_database,
    tables=["albums", "tracks", "artists"],
    verbose=True,
)
sql_tool = QueryEngineTool.from_defaults(
    query_engine=sql_query_engine,
    name="sql_tool",
    description=(
        "Useful for translating a natural language query into a SQL query"
    ),
)

#####Define Agent Input Component

In [8]:
from llama_index.core.agent.react.types import (
    ActionReasoningStep,
    ObservationReasoningStep,
    ResponseReasoningStep,
)
from llama_index.core.agent import Task, AgentChatResponse
from llama_index.core.query_pipeline import (
    StatefulFnComponent,
    QueryComponent,
    ToolRunnerComponent,
)
from llama_index.core.llms import MessageRole
from typing import Dict, Any, Optional, Tuple, List, cast


# Input Component
## This is the component that produces agent inputs to the rest of the components
## Can also put initialization logic here.
def agent_input_fn(state: Dict[str, Any]) -> str:
    """Agent input function.

    Returns:
        A Dictionary of output keys and values. If you are specifying
        src_key when defining links between this component and other
        components, make sure the src_key matches the specified output_key.

    """
    task = state["task"]
    if len(state["current_reasoning"]) == 0:
        reasoning_step = ObservationReasoningStep(observation=task.input)
        state["current_reasoning"].append(reasoning_step)
    return task.input


agent_input_component = StatefulFnComponent(fn=agent_input_fn)

#####Define Agent Prompt

In [9]:
from llama_index.core.agent import ReActChatFormatter
from llama_index.core.query_pipeline import InputComponent, Link
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool


## define prompt function
def react_prompt_fn(
    state: Dict[str, Any], input: str, tools: List[BaseTool]
) -> List[ChatMessage]:
    task = state["task"]
    # Add input to reasoning
    chat_formatter = ReActChatFormatter()
    cur_prompt = chat_formatter.format(
        tools,
        chat_history=task.memory.get(),
        current_reasoning=state["current_reasoning"],
    )
    return cur_prompt


react_prompt_component = StatefulFnComponent(
    fn=react_prompt_fn, partial_dict={"tools": [sql_tool]}
)

#####Define Agent Output Parser + Tool Pipeline

In [10]:
from typing import Set, Optional
from llama_index.core.agent.react.output_parser import ReActOutputParser
from llama_index.core.llms import ChatResponse
from llama_index.core.agent.types import Task


def parse_react_output_fn(state: Dict[str, Any], chat_response: ChatResponse):
    """Parse ReAct output into a reasoning step."""
    output_parser = ReActOutputParser()
    reasoning_step = output_parser.parse(chat_response.message.content)
    return {"done": reasoning_step.is_done, "reasoning_step": reasoning_step}


parse_react_output = StatefulFnComponent(fn=parse_react_output_fn)


def run_tool_fn(state: Dict[str, Any], reasoning_step: ActionReasoningStep):
    """Run tool and process tool output."""
    task = state["task"]
    tool_runner_component = ToolRunnerComponent(
        [sql_tool]
    )
    tool_output = tool_runner_component.run_component(
        tool_name=reasoning_step.action,
        tool_input=reasoning_step.action_input,
    )
    observation_step = ObservationReasoningStep(observation=str(tool_output))
    state["current_reasoning"].append(observation_step)
    # TODO: get output

    # return tuple of current output and False for is_done
    return observation_step.get_content(), False


run_tool = StatefulFnComponent(fn=run_tool_fn)


def process_response_fn(
    state: Dict[str, Any], response_step: ResponseReasoningStep
):
    """Process response."""
    state["current_reasoning"].append(response_step)
    return response_step.response, True


process_response = StatefulFnComponent(fn=process_response_fn)

#####Stitch together Agent Query Pipeline


In [11]:
from llama_index.core.query_pipeline import QueryPipeline as QP
from llama_index.llms.openai import OpenAI

qp.add_modules(
    {
        "agent_input": agent_input_component,
        "react_prompt": react_prompt_component,
        "llm": OpenAI(model="gpt-4o-mini"),
        "react_output_parser": parse_react_output,
        "run_tool": run_tool,
        "process_response": process_response,
    }
)

In [12]:
# link input to react prompt to parsed out response (either tool action/input or observation)
qp.add_chain(["agent_input", "react_prompt", "llm", "react_output_parser"])

# add conditional link from react output to tool call (if not done)
qp.add_link(
    "react_output_parser",
    "run_tool",
    condition_fn=lambda x: not x["done"],
    input_fn=lambda x: x["reasoning_step"],
)
# add conditional link from react output to final response processing (if done)
qp.add_link(
    "react_output_parser",
    "process_response",
    condition_fn=lambda x: x["done"],
    input_fn=lambda x: x["reasoning_step"],
)

#####Setup Agent Worker around Text-to-SQL Query Pipeline

In [13]:
from llama_index.core.agent import FnAgentWorker
from typing import Dict, Tuple, Any


def run_agent_fn(state: Dict[str, Any]) -> Tuple[Dict[str, Any], bool]:
    """Run agent function."""
    task, qp = state["__task__"], state["query_pipeline"]
    # if first run, then set query pipeline state to initial variables
    if state["is_first"]:
        qp.set_state(
            {
                "task": task,
                "current_reasoning": [],
            }
        )
        state["is_first"] = False

    # no explicit input here, just run root node
    response_str, is_done = qp.run()
    # if done, store output and log to memory
    # a core memory module is available in the `task` variable. Of course you can log
    # and store your own memory as well
    state["__output__"] = response_str
    if is_done:
        task.memory.put_messages(
            [
                ChatMessage(content=task.input, role=MessageRole.USER),
                ChatMessage(content=response_str, role=MessageRole.ASSISTANT),
            ]
        )
    return state, is_done


agent = FnAgentWorker(
    fn=run_agent_fn,
    initial_state={"query_pipeline": qp, "is_first": True},
).as_agent()

#####Run the Agent

In [14]:
# start task
task = agent.create_task(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

In [15]:
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {"input":"List 3 tracks from the artist AC/DC"}

> Running module run_tool with input: 
reasoning_step: thought='The current language of the user is: English. I need to use a tool to help me answer the question.' action='sql_tool' action_input={'input': 'List 3 tracks from the artist AC/DC'}

> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), Ar

In [16]:
step_output = agent.run_step(task.task_id)

> Running module agent_input with input: 

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Tres canciones del artista AC/DC son "For Those About To Rock (We Salute You)", "Put The ...

> Running module process_response with input: 
response_step: thought="I can answer without using any more tools. I'll use the user's language to answer." response='Tres canciones del artista AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger O...



In [17]:
step_output.is_last

True

In [18]:
response = agent.finalize_response(task.task_id)

In [19]:
print(str(response))

Tres canciones del artista AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger On You" y "Let's Get It Up".


In [21]:
# run this e2e
sql_agent = agent
sql_agent.reset()
response = sql_agent.chat(
    "What are some tracks from the artist AC/DC? Limit it to 3"
)

> Running module agent_input with input: 

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {"input":"List 3 tracks from the artist AC/DC"}

> Running module run_tool with input: 
reasoning_step: thought='The current language of the user is: English. I need to use a tool to help me answer the question.' action='sql_tool' action_input={'input': 'List 3 tracks from the artist AC/DC'}

> Table desc str: Table 'albums' has columns: AlbumId (INTEGER), Title (NVARCHAR(160)), Ar

> Running module agent_input with input: 

> Running module react_prompt with input: 
input: What are some tracks from the artist AC/DC? Limit it to 3

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Tres canciones del artista AC/DC son "For Those About To Rock (We Salute You)", "Put The ...

> Running module process_response with input: 
response_step: thought="I can answer without using any more tools. I'll use the user's language to answer." response='Tres canciones del artista AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger O...



In [22]:
print(str(response))

Tres canciones del artista AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger On You" y "Let's Get It Up".


In [31]:
from llama_index.core.tools import QueryEngineTool

query_engine_tools = [
    QueryEngineTool(
        query_engine=sql_agent,
        metadata=ToolMetadata(
            name="sql_agent", description="Agent that can execute SQL queries."
        ),
    ),
    #QueryEngineTool(
    #    query_engine=gmail_agent,
    #    metadata=ToolMetadata(
    #        name="gmail_agent",
    #        description="Tool that can send emails on Gmail.",
    #    ),
    #),
]

outer_agent = ReActAgent.from_tools(query_engine_tools, llm=llm, verbose=True)

In [33]:
outer_agent.chat("What are some tracks from the artist AC/DC? Limit it to 3. Use the database")

> Running step 24170a1a-f077-400b-9025-08adb8be3435. Step input: What are some tracks from the artist AC/DC? Limit it to 3. Use the database


Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: sql_agent
Action Input: {'input': "SELECT track_name FROM tracks WHERE artist_name = 'AC/DC' LIMIT 3;"}
> Running module agent_input with input: 

> Running module react_prompt with input: 
input: SELECT track_name FROM tracks WHERE artist_name = 'AC/DC' LIMIT 3;

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: The current language of the user is: English. I need to use a tool to help me answer the question.
Action: sql_tool
Action Input: {"input":"SELECT track_name FROM tracks WHERE arti...

> Running module run_tool with input: 
reasoning_step: thought='The current language of the user is:

> Running module agent_input with input: 

> Running module react_prompt with input: 
input: SELECT track_name FROM tracks WHERE artist_name = 'AC/DC' LIMIT 3;

> Running module llm with input: 
messages: [ChatMessage(role=<MessageRole.SYSTEM: 'system'>, additional_kwargs={}, blocks=[TextBlock(block_type='text', text='You are designed to help with a variety of tasks, from answering questions to providi...

> Running module react_output_parser with input: 
chat_response: assistant: Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: Las tres mejores canciones de AC/DC son "For Those About To Rock (We Salute You)", "Put T...

> Running module process_response with input: 
response_step: thought="I can answer without using any more tools. I'll use the user's language to answer." response='Las tres mejores canciones de AC/DC son "For Those About To Rock (We Salute You)", "Put The Finge...

Observation: Las tres mejores canciones de AC/DC son 

AgentChatResponse(response='Las tres mejores canciones de AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger On You" y "Let\'s Get It Up".', sources=[ToolOutput(content='Las tres mejores canciones de AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger On You" y "Let\'s Get It Up".', tool_name='sql_agent', raw_input={'input': "SELECT track_name FROM tracks WHERE artist_name = 'AC/DC' LIMIT 3;"}, raw_output=Response(response='Las tres mejores canciones de AC/DC son "For Those About To Rock (We Salute You)", "Put The Finger On You" y "Let\'s Get It Up".', source_nodes=[], metadata=None), is_error=False)], source_nodes=[], is_dummy_stream=False, metadata=None)

###Agent With Planning

In [42]:
from llama_index.agent.openai import OpenAIAgentWorker
from llama_index.core.agent import (
    StructuredPlannerAgent,
    FunctionCallingAgentWorker,
)

lyft_tool = QueryEngineTool.from_defaults(
    lyft_index.as_query_engine(),
    name="lyft_2021",
    description="Useful for asking questions about Lyft's 2021 10-K filling.",
)

uber_tool = QueryEngineTool.from_defaults(
    uber_index.as_query_engine(),
    name="uber_2021",
    description="Useful for asking questions about Uber's 2021 10-K filling.",
)

worker = FunctionCallingAgentWorker.from_tools(
    [lyft_tool, uber_tool], verbose=True
)

agent = StructuredPlannerAgent(
    worker, tools=[lyft_tool, uber_tool], verbose=True
)

In [44]:
import nest_asyncio

nest_asyncio.apply()

In [45]:
response = agent.chat(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

=== Initial plan ===
Ask about Lyft's key risk factors in 2021 10-K filing:
Ask about the key risk factors in Lyft's 2021 10-K filing -> Key risk factors for Lyft in 2021 10-K filing
deps: []


Ask about Uber's key risk factors in 2021 10-K filing:
Ask about the key risk factors in Uber's 2021 10-K filing -> Key risk factors for Uber in 2021 10-K filing
deps: []


> Running step ffac3609-34ce-45c5-875a-5221afefec27. Step input: Ask about the key risk factors in Lyft's 2021 10-K filing
Added user message to memory: Ask about the key risk factors in Lyft's 2021 10-K filing
> Running step f2ceb826-954e-4128-ae15-c30d3931c949. Step input: Ask about the key risk factors in Uber's 2021 10-K filing
Added user message to memory: Ask about the key risk factors in Uber's 2021 10-K filing
=== Calling Function ===
Calling function: uber_2021 with args: {"input": "key risk factors"}
=== Calling Function ===
Calling function: lyft_2021 with args: {"input": "key risk factors"}
=== Function Output ===

###Lower-Level API

In [46]:
from llama_index.core.agent import AgentRunner
from llama_index.agent.openai import OpenAIAgentWorker

openai_step_engine = OpenAIAgentWorker.from_tools([lyft_tool, uber_tool], llm=llm, verbose=True)
agent = AgentRunner(openai_step_engine)

In [47]:
response = agent.chat(
    "Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings."
)

Added user message to memory: Summarize the key risk factors for Lyft and Uber in their 2021 10-K filings.
=== Calling Function ===
Calling function: lyft_2021 with args: {"input": "key risk factors"}
Got output: The key risk factors outlined in the provided context include general economic factors, operational factors, regulatory and legal factors, financing and transactional risks, as well as governance risks and risks related to ownership of the company's capital stock.

=== Calling Function ===
Calling function: uber_2021 with args: {"input": "key risk factors"}
Got output: The key risk factors include potential challenges related to autonomous vehicle technologies, retention of high-quality personnel, security and data privacy breaches, cyberattacks, climate change risks, reliance on third parties for distribution and software, need for additional capital, risks associated with acquisitions and integration, legal and regulatory risks, risks related to data processing, intellectual